<a href="https://colab.research.google.com/github/gwhagerty/Hackathon/blob/main/nyu_lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
name = 'hungry'
print("hello world! i'm " + name)

In [ ]:
# @title Hidden code for default chart styles
# plotting specific stuff to ignore
import matplotlib
import warnings
warnings.filterwarnings('ignore')
font = {'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)
matplotlib.rcParams['figure.figsize'] = (12, 6)

In [ ]:
import pandas as pd

raw_data = pd.read_csv('https://raw.githubusercontent.com/jdanial18/energy_markets_forecasting/main/data.csv')
raw_data.head(5)

In [ ]:
# this section makes a "datetime" index
# it will help us with plotting and other things
# but isnt needed for forecasts
data = raw_data.set_index('datetime')
data.index = pd.to_datetime(data.index)
data.head()

In [ ]:
# line plot
data.plot(y='load')

In [ ]:
data = data.resample('D').mean().dropna()
data.head()

In [ ]:
data.plot(y='load')
data.plot(y='temp', color='r')

In [ ]:
data.plot(y='load')
data.plot(y='temp', color='r')

In [ ]:
data.plot.scatter(x='temp', y='load', alpha=.5)

In [ ]:
import statsmodels.api as sm
data['constant'] = 1
model = sm.OLS(data['load'], data[['temp', 'constant']])
result = model.fit()
data['prediction'] = result.predict(data[['temp', 'constant']])
ax = data.plot.scatter('temp', 'load')
data.plot(ax=ax, x='temp', y='prediction', color='r')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
# x vs y this time
model = model.fit(data[['temp']], data['load'])
data['pred'] = model.predict(data[['temp']])
ax = data.plot.scatter('temp', 'load')
data.plot.scatter(ax=ax, x='temp', y='pred', color='r')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
xgb = GradientBoostingRegressor()
ada = AdaBoostRegressor()
train = data[data.index < '1/1/2019']
test = data[data.index > '1/1/2019']
x_train = train[['temp']]
y_train = train[['load']]
x_test = test[['temp']]
y_test = test[['load']]
xgb = xgb.fit(x_train, y_train)
ada = ada.fit(x_train, y_train)
y_test['xgb'] = xgb.predict(x_test)
y_test['ada'] = ada.predict(x_test)
ax = train.plot(y='load')
y_test.plot(ax=ax)

In [ ]:
data['day'] = data.index.dayofweek
data['month'] = data.index.month
data.head()

In [ ]:
day_dum = pd.get_dummies(data['day'], prefix='day')
month_dum = pd.get_dummies(data['month'], prefix='month')
month_dum.head()

In [ ]:
day_dum_cols = day_dum.columns
month_dum_cols = month_dum.columns

data = pd.concat([data, day_dum], 1)
data = pd.concat([data, month_dum], 1)

In [ ]:
x_columns = ['temp'] + list(day_dum_cols) + list(month_dum_cols)

xgb = GradientBoostingRegressor()
ada = AdaBoostRegressor()
train = data[data.index < '1/1/2018']
test = data[data.index > '1/1/2018']


x_train = train[x_columns]
y_train = train[['load']]
x_test = test[x_columns]
y_test = test[['load']]
xgb = xgb.fit(x_train, y_train)
ada = ada.fit(x_train, y_train)
y_test['xgb'] = xgb.predict(x_test)
# y_test['ada'] = ada.predict(x_test)
ax = train.plot(y='load')
y_test.plot(ax=ax)

In [ ]:
!pip install meteostat
import pandas as pd
import meteostat as met

def load_data():
    url = 'https://www.eia.gov/opendata/qb.php?category=3389943&sdid=EBA.NY-ALL.D.HL'
    load = pd.read_html(url)[0]
    load.columns = load.columns.str.lower().str.replace(' ', '_')
    load['datetime'] = pd.to_datetime(load['period'].str.split('-').str[0])
    load = load.set_index('datetime')[['value']]
    load.columns = ['load']
    load = load.resample('H').mean()

    stations = met.Stations()
    inv = stations.inventory(freq='hourly').fetch()
    ny = inv[(inv['country'] == 'US') & (inv['region'] == 'NY')]
    nyc = ny[ny['name'] == 'LaGuardia Airport']
    nyc = nyc.reset_index().loc[0]
    weather = met.Hourly(nyc['id'],
                        start=load.index.min(),
                        end=load.index.max(),
                        timezone=nyc['timezone'])
    weather = weather.fetch()
    weather.index = pd.to_datetime(weather.index.astype(str).str[:-6])
    weather = weather.resample('H').mean()
    df = pd.concat([load, weather[['temp']]], 1).dropna(0)
    return df 
